# <span style="font-family:impact; font-size:2.5em;color:deepskyblue">Entrenamiento Used Cars Dataset</span>

### <span style="font-family:impact; font-size:1.75em;color:royalblue">Proyecto de Open Data II</span>

## <span style="font-family:impact; font-size:1em;color:mediumblue">Elena Delgado y Javier García</span>

___

# <span style="font-family:monospace;color:coral">ÍNDICE</span>
1. Recordatorio de nuestro proyecto
2. Preparación Datos
3. SQL
4. Learning method
    + División de datos
    - Pasos a seguir para crear predicciones
    + Random Forest
    - Gradient Boostes Tree Regressor
    + Linear Regressor
    - Grid Search con Random Forest
    + Discretizar Variables Continuas
    - Linear Regressor Disretized
    + Estandarizar Variables Continuas
    - Linear Regression Standarized
    + PCA
    - Comparación Final de Métodos
5. Conclusiones 

___

In [1]:
import pandas as pd 
import numpy as np                         
%matplotlib inline
import findspark
findspark.init()
findspark.find()
import pyspark
import pyspark.sql.types as typ
from pyspark.sql.types import *
from pyspark.sql.session import SparkSession
import pyspark.sql.functions as func
from pyspark.sql.functions import col

In [2]:
import os
exec(open(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.4
      /_/

Using Python version 3.7.3 (default, Mar 27 2019 16:54:48)
SparkSession available as 'spark'.


## <span style="font-family:monospace;color:salmon">RECORDATORIO DE NUESTRO PROYECTO</span>

_Used Cars Dataset_ es un dataset que reune todas las ventas de vehículos de segunda mano dentro de Estados Unidos desde el año 1990 a 2019.

![.](https://lh5.googleusercontent.com/proxy/acYCTVi9zclt2LOeL2fNcXAHhSmLTgbMDq1ZeHlnUt9zXouj_f1MThWSgYvXIsvIp-4tWmgF9YoyuHYL3jG74FPHdaIlPGMa2KQlajF0pvrBxWgxe3Hej1IBS4GU8By0U8g1YBtAxhM)

Este dataset tiene las siguientes columnas:
+ city
+ price
+ year
+ manufacturer
+ make
+ condition
+ cylinders
+ fuel
+ odometer
+ transmission
+ drive
+ type
+ paint_color

___

# <span style="font-family:monospace;color:RED">PREPARACIÓN DE DATOS</span>

## <span style="font-family:monospace;color:orangered">Exportamos CSV</span>

Para comenzar, vamos a importar dos csv con spark.
1. Coches_db es el csv final que obtuvimos en Proyecto de Open Data I donde todas las variables son numéricas excepto la columna de modelo de coche _make_. 
2. Coches1_db es el dataset que contiene variables categóricas y numéricas

In [3]:
coches_db = spark.read.csv('./coches.csv', header='true', inferSchema='true', sep=',') 
coches_db.createOrReplaceTempView("coche")

In [4]:
coches1_db = spark.read.csv('./coches1.csv', header='true', inferSchema='true', sep=',') 
coches1_db.createOrReplaceTempView("coche")

Vamos a visualizar las columnas que tenia nuestro dataset

In [5]:
coches_db.head(1)

[Row(_c0=0, city=1, price=9000, year=2009.0, manufacturer=2, make='suburban lt2', condition='1', cylinders=8, fuel=1, odometer=217743.0, transmission=1.0, drive=3, type=2, paint_color=2)]

## <span style="font-family:monospace;color:coral">Creamos el esquema</span>

Vamos a coger todas las columnas de nuestro dataset para hacer el **esquema** que usaremos más tarde para entrenar nuestro modelo

In [6]:
schema = StructType([
    StructField("_c0", IntegerType(), True),
    StructField("city", IntegerType(), True),
    StructField("price", IntegerType(), True),
    StructField("year", DoubleType(), True),
    StructField("manufacturer", IntegerType(), True),
    StructField("make", StringType(), True),
    StructField("condition", StringType(), True),
    StructField("cylinders", IntegerType(), True),
    StructField("fuel", IntegerType(), True),
    StructField("odometer", DoubleType(), True),
    StructField("transmission", DoubleType(), True),
    StructField("drive", IntegerType(), True),
    StructField("type", IntegerType(), True),
    StructField("paint_color", IntegerType(), True),
])

In [7]:
coches_df = spark.read.csv('coches.csv', header='true', inferSchema='false', schema=schema, sep=',')

Ahora vamos a visualizar el esquema para ver qué datos son necesarios y cuales no. Además nos va a servir para ver que se ha implementado correctamente.

In [8]:
coches_df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- city: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- year: double (nullable = true)
 |-- manufacturer: integer (nullable = true)
 |-- make: string (nullable = true)
 |-- condition: string (nullable = true)
 |-- cylinders: integer (nullable = true)
 |-- fuel: integer (nullable = true)
 |-- odometer: double (nullable = true)
 |-- transmission: double (nullable = true)
 |-- drive: integer (nullable = true)
 |-- type: integer (nullable = true)
 |-- paint_color: integer (nullable = true)



Vamos a seleccionar columnas que queremos usar más tarde para la predicción y las metemos en **_cochesnuevo_**

In [9]:
cochesnuevo=coches_df.select("price","city","year","manufacturer","cylinders","fuel","odometer","transmission","drive","type","paint_color")

In [10]:
type(cochesnuevo)

pyspark.sql.dataframe.DataFrame

## <span style="font-family:monospace;color:orange">Modificaciones de coches1_db</span>

Vamos a **eliminar** la columna todas las columnas numericas coches1_db ya que no nos van a ser utiles, y vamos a **renombrar** las columnas categóricas para que sea más sencillo de buscar.

In [11]:
columns_to_drop = ['_c0']
coches1_db = coches1_db.drop(*columns_to_drop)
columns_to_drop = ['price']
coches1_db = coches1_db.drop(*columns_to_drop)
columns_to_drop = ['year']
coches1_db = coches1_db.drop(*columns_to_drop)
columns_to_drop = ['cylinders']
coches1_db = coches1_db.drop(*columns_to_drop)
columns_to_drop = ['odometer']
coches1_db = coches1_db.drop(*columns_to_drop)
columns_to_drop = ['make']
coches1_db = coches1_db.drop(*columns_to_drop)

In [12]:
coches1_db=coches1_db.withColumnRenamed('city','state')
coches1_db=coches1_db.withColumnRenamed('manufacturer','manufacturerN')
coches1_db=coches1_db.withColumnRenamed('condition','conditionN')
coches1_db=coches1_db.withColumnRenamed('fuel','fuelN')
coches1_db=coches1_db.withColumnRenamed('transmission','transmissionN')
coches1_db=coches1_db.withColumnRenamed('drive','driveN')
coches1_db=coches1_db.withColumnRenamed('type','typeN')
coches1_db=coches1_db.withColumnRenamed('paint_color','paint_colorN')

Vamos a visualizar las **columnas** que han quedado en el segundo Dataset para ver que las modificaciones que hemos implementado se han completado. 

In [13]:
coches1_db.columns

['state',
 'manufacturerN',
 'conditionN',
 'fuelN',
 'transmissionN',
 'driveN',
 'typeN',
 'paint_colorN']

## <span style="font-family:monospace;color:gold">Visualizamos las tablas</span>

Vamos a visualizar las dos tablas para que nos facilite nuestro trabajo en caso de que necesitemos visualizar algún dato rapidamente o consultar el tipo de los datos

In [14]:
cochesnuevo.show(10)

+-----+----+------+------------+---------+----+------------------+------------+-----+----+-----------+
|price|city|  year|manufacturer|cylinders|fuel|          odometer|transmission|drive|type|paint_color|
+-----+----+------+------------+---------+----+------------------+------------+-----+----+-----------+
| 9000|   1|2009.0|           2|        8|   1|          217743.0|         1.0|    3|   2|          2|
|31999|   1|2012.0|           6|        6|   2|111093.57002957871|         1.0|    1|   2|          3|
|16990|   1|2003.0|           6|        6|   2| 153430.5303265941|         2.0|    1|   2|          1|
| 6000|   1|2002.0|           8|        8|   1|          195000.0|         1.0|    1|   4|          2|
|37000|   1|2012.0|           2|        8|   2|          178000.0|         1.0|    1|   4|          3|
| 3700|   1|2003.0|           2|        8|   1|          269000.0|         1.0|    1|   4|          3|
|19950|   1|2013.0|           1|        8|   1|          116792.0|       

In [15]:
coches1_db.show(10)

+-----+-------------+----------+------+-------------+------+------+------------+
|state|manufacturerN|conditionN| fuelN|transmissionN|driveN| typeN|paint_colorN|
+-----+-------------+----------+------+-------------+------+------+------------+
|   TX|    chevrolet|      good|   gas|    automatic|   rwd|   SUV|       white|
|   TX|          ram| excellent|diesel|    automatic|   4wd|   SUV|      silver|
|   TX|          ram|      good|diesel|       manual|   4wd|   SUV|       black|
|   TX|          gmc|      good|   gas|    automatic|   4wd|pickup|       white|
|   TX|    chevrolet| excellent|diesel|    automatic|   4wd|pickup|      silver|
|   TX|    chevrolet|      fair|   gas|    automatic|   4wd|pickup|      silver|
|   TX|         ford| excellent|   gas|    automatic|   4wd|pickup|       white|
|   TX|          ram| excellent|diesel|       manual|   4wd|   SUV|      silver|
|   TX|         ford| excellent|diesel|    automatic|   4wd|pickup|       white|
|   TX|         ford| excell

# <span style="font-family:monospace;color:chartreuse">SQL</span>

![.](https://lh3.googleusercontent.com/proxy/Pl8zlhBktOGY9YLi2u67L_OGp33oQx0ToxJZdZSKLFAX1blDBScEbNZbp7u1pgC0tkXdsBy4n7YWEmdeDxh0O_Awa3rp5XHIbG2bgtwFMaawEAeiZ5qLiN3lUq2ap_SwSq_mtnMUOYgu6d_FrC1aX4qMpyoMqQ)

Vamos a crear tablas *SQL* para poder visualizar nuestros datos.

In [16]:
cochesnuevo.createOrReplaceTempView("cochesnuevo_tb")

In [17]:
coches1_db.createOrReplaceTempView("coches1_tb")

Ahora vamos a visualizar cuantas filas tenemos en ambos datasets, para saber que no se ha modificado nada anteriormente y ver con cuantos datos trabajamos.

In [18]:
spark.sql("select count(*) from cochesnuevo_tb").collect()

[Row(count(1)=417197)]

In [19]:
spark.sql("select count(*) from coches1_tb").collect()

[Row(count(1)=417197)]

Vamos a visualizar los colores de coches disintos que tenemos al igual que los diferentes estados de nuestros datos, para que ver que todo sigue bien.

In [20]:
spark.sql('select distinct(paint_colorN) from coches1_tb').collect()

[Row(paint_colorN='orange'),
 Row(paint_colorN='grey'),
 Row(paint_colorN='green'),
 Row(paint_colorN='yellow'),
 Row(paint_colorN='silver'),
 Row(paint_colorN='purple'),
 Row(paint_colorN='white'),
 Row(paint_colorN='red'),
 Row(paint_colorN='custom'),
 Row(paint_colorN='black'),
 Row(paint_colorN='brown'),
 Row(paint_colorN='truck'),
 Row(paint_colorN='blue')]

In [21]:
coches1_db.select('state').distinct().show()

+-----+
|state|
+-----+
|   SC|
|   AZ|
|   LA|
|   MN|
|   NJ|
|   DC|
|   OR|
|   VA|
|   RI|
|   KY|
|   WY|
|   NH|
|   MI|
|   NV|
|   WI|
|   ID|
|   CA|
|   CT|
|   NE|
|   MT|
+-----+
only showing top 20 rows



Una vez tenemos todos los Datasets revisados, vamos a entrenar nuestro modelo. Como nuestro Dataset no fue sacado de los challenges y nuestro proposito desde que empezamos a manejar los datos fue predecir el precio futuro de un coche, ahora vamos a estudiar con métodos de predicción adecuados los precios.

___

# <span style="font-family:monospace;color:turquoise">LEARNING METHOD</span>

![.](https://images.squarespace-cdn.com/content/5be4713f70e80254dd35b8b3/1549656674295-0WPM8V4DZ97XGPR01S13/Machine-learning-BG.jpeg?format=1500w&content-type=image%2Fjpeg)

In [22]:
import pyspark.ml.feature as ft
import pyspark.ml.regression as rg
from pyspark.ml import Pipeline
import pyspark.ml.evaluation as ev
from pyspark.ml.feature import VectorAssembler, VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.param import Param, Params

Un modelo de regresión es un modelo matemático que busca determinar la relación entre una variable dependiente (Y) con respecto a otras variables llamadas explicativas o independientes (X). Asimismo, el modelo busca determinar cuál será el impacto sobre la variable Y ante un cambio en las variables explicativas (X).

Puesto que nosotros tenemos que hacer una regresión, vamos a ver que métodos de entrenamiento teneos. 
+ Linear regression
+ Decision tree regression
+ Random forest regression
+ Gradient-boosted tree regression
+ Isotonic regression

## <span style="font-family:monospace;color:deepskyblue">División de datos</span>

Vamos a dividir nuestros datos, 80 % va a ser de los datos de entrenamiento y el otro 20 % es para los datos de test. Ambos que luego se usaran para entrenar el modelo

In [23]:
# Split the data into training and test sets (20% held out for testing)
(trainingData, testData) = cochesnuevo.randomSplit([0.8, 0.2])

Antes de comenzar, vamos a pasar nuestra columna de price para que sea de DoubleType y así podamos usarla en los modelos.

In [24]:
cochesnuevo = cochesnuevo.withColumn('price', cochesnuevo['price'].cast(typ.DoubleType()))

## <span style="font-family:monospace;color:royalblue">Pasos a seguir para crear predicciones con nuestros algoritmos</span>

1. Para realizar este algoritmo vamos a empezar creando una columna que reúna todas las variables utilizando los **transformadores**.
2. Preparar nuestro **modelo** con sus respectivos valores en caso de tener.
3. Creamos el **pipeline**. El Pipeline nos va a ayudar a meter nuestros transformadores y el modelo para ejecutarlo uno detrás de otro.
4. **Modelamos** nuestros datos de training el cual tiene el 80% de nuestros datos.
5. A continuación **testeamos** el modelo 
6. **Visualizamos** las predicciones que hemos obtenido.
7. **Evaluamos** el modelo 
8. Calculamos el **RMSE** (raíz del error cuadrático medio) y el **MSE** (error cuadrático medio)  de nuestro modelo. 


___

## <span style="font-family:monospace;color:mediumslateblue">RANDOM FOREST</span>

In [25]:
from pyspark.ml.regression import RandomForestRegressor

**¿Es *Random Forest* un algoritmo apropiado para nuestros datos?**

![.](https://www.paradigmadigital.com/wp-content/uploads/2017/03/machine-learning-dummies-6.png)

El primer algoritmo que decidimos utilizar fue Random Forest ya que este algoritmo de regresión consiste en un conjunto de múltiples árboles de decisión independientes, al cual se le asigna un conjunto de datos aleatorios con una misma distribución. La salida es el promedio de los resultados finales de cada árbol.

Vamos a empezar creando una columna que reúna todas las variables utilizando el transformador **VectorAssembler**.

In [26]:
mivectorRF = ft.VectorAssembler(
    inputCols=["city","year","manufacturer","cylinders","fuel","odometer","transmission","drive","type","paint_color"], 
    outputCol='features'
)

+ Ahora vamos a preparar nuestro modelo con sus valores:
    + 5 árboles para entrenar nuestros datos
    + 5 niveles de profundidad de los árboles
    + 'price', la columna que tiene que predecir

In [27]:
algoritmoRF = rg.RandomForestRegressor(
    numTrees=5, 
    maxDepth=5, 
    labelCol='price')

Utilizamos el **Pipeline** ya que nos va a ayudar a meter nuestro VectorAssembler y el modelo para ejecutarlo uno detrás de otro.

In [28]:
pipelineRF = Pipeline(stages=[mivectorRF, algoritmoRF])

**Modelamos** nuestros datos de **training** el cual tiene el 80% de nuestros datos.

In [29]:
modelRF = pipelineRF.fit(trainingData)

Ahora vamos a poner al modelo a que haga las **prediciones** con los datos.

In [30]:
testRF = modelRF.transform(testData)

**Visualizamos** las dos primeras columnas para ver que prediccion nos ha hecho nuestro modelo.

In [31]:
testRF.take(2)

[Row(price=2050, city=5, year=1999.0, manufacturer=3, cylinders=6, fuel=1, odometer=202000.0, transmission=1.0, drive=2, type=1, paint_color=2, features=DenseVector([5.0, 1999.0, 3.0, 6.0, 1.0, 202000.0, 1.0, 2.0, 1.0, 2.0]), prediction=5173.5504671237995),
 Row(price=2100, city=2, year=1996.0, manufacturer=1, cylinders=6, fuel=1, odometer=151235.29235237173, transmission=1.0, drive=1, type=1, paint_color=2, features=DenseVector([2.0, 1996.0, 1.0, 6.0, 1.0, 151235.2924, 1.0, 1.0, 1.0, 2.0]), prediction=5173.5504671237995)]

Ahora vamos a **evaluar** nuestro modelo para ver la raíz del error cuadratico medio ***RMSE***, y el error cuadrático medio ***MSE***. 

In [32]:
evaluator = RegressionEvaluator(
    labelCol="price", predictionCol="prediction", metricName="rmse")
rmseRF = evaluator.evaluate(testRF)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmseRF) 

Root Mean Squared Error (RMSE) on test data = 10753.2


In [33]:
mseRF = rmseRF*rmseRF
print("Mean Squared Error (MSE) on test data = %g" % mseRF) 

Mean Squared Error (MSE) on test data = 1.15632e+08


Contestando a la pregunta incial, podemos ver que Random Forest tiene un alto error cuadrático medio para nuestro modelo, por lo que **NO** ha realizado una buena predicción.

___

## <span style="font-family:monospace;color:darkorchid">GRADIENT BOOSTED TREE REGRESSOR</span>

In [34]:
from pyspark.ml.regression import GBTRegressor

**¿Es _Gradient Boosted Tree Regressor_ un algoritmo apropiado para nuestros datos?**

![.](https://littleml.files.wordpress.com/2017/03/boosted-trees-process.png)

Gradient Boosted Tree Regressor es una técnica de aprendizaje automático que produce un modelo de predicción en forma de un conjunto de modelos de predicción débiles, típicamente árboles de decisión. En estos casos, construye cada árbol de regresión de forma gradual, utilizando una función de pérdida predefinida para medir el error en cada paso y corregirlo en el siguiente.

Habiendo visto que Random Forest no es un modelo adecuado para nuestros datos, hemos decidido comprobar si este modelo nos proporciona mejores predicciones. 

Al igual que con el método anterior, vamos a empezar creando una columna que reúna todas las variables utilizando **transformadores**.

In [35]:
featuresCols = cochesnuevo.columns

In [36]:
mivectorGBT = VectorAssembler(inputCols=featuresCols, outputCol="rawFeatures")

In [37]:
# This identifies categorical features and indexes them.
vectorIndexer = VectorIndexer(inputCol="rawFeatures", outputCol="features", maxCategories=4)

Preparamos nuestro **modelo** con columna a predecir 'price'

In [38]:
gbt = GBTRegressor(labelCol="price")

Creamos el **pipeline**  para ejecutar nuestro VectorAssembler, el VectorIndexer y el modelo uno detrás de otro.

In [39]:
pipelineGBT = Pipeline(stages=[mivectorGBT,vectorIndexer, gbt])

Realizamos el **modelado** de nuestros datos de training (80% de los datos). 

In [40]:
modelGBT = pipelineGBT.fit(trainingData)

**Entrenamos** el modelo con los datos de test (20%)

In [41]:
testGBT = modelGBT.transform(testData)

**Visualizamos** las tres primeras prediciones para ver como ha funcionado nuestro modelo

In [42]:
testGBT.take(3)

[Row(price=2050, city=5, year=1999.0, manufacturer=3, cylinders=6, fuel=1, odometer=202000.0, transmission=1.0, drive=2, type=1, paint_color=2, rawFeatures=DenseVector([2050.0, 5.0, 1999.0, 3.0, 6.0, 1.0, 202000.0, 1.0, 2.0, 1.0, 2.0]), features=DenseVector([2050.0, 5.0, 1999.0, 3.0, 6.0, 1.0, 202000.0, 1.0, 1.0, 1.0, 2.0]), prediction=2429.1004923032024),
 Row(price=2100, city=2, year=1996.0, manufacturer=1, cylinders=6, fuel=1, odometer=151235.29235237173, transmission=1.0, drive=1, type=1, paint_color=2, rawFeatures=DenseVector([2100.0, 2.0, 1996.0, 1.0, 6.0, 1.0, 151235.2924, 1.0, 1.0, 1.0, 2.0]), features=DenseVector([2100.0, 2.0, 1996.0, 1.0, 6.0, 1.0, 151235.2924, 1.0, 0.0, 1.0, 2.0]), prediction=2429.1004923032024),
 Row(price=2100, city=2, year=1999.0, manufacturer=5, cylinders=4, fuel=1, odometer=162695.9298187809, transmission=1.0, drive=1, type=6, paint_color=1, rawFeatures=DenseVector([2100.0, 2.0, 1999.0, 5.0, 4.0, 1.0, 162695.9298, 1.0, 1.0, 6.0, 1.0]), features=DenseVec

**Evaluamos** la precisión de predicción de nuestro modelo con un _RegressionEvaluator_

In [43]:
evaluator = RegressionEvaluator(
    labelCol="price", predictionCol="prediction", metricName="rmse")

Calculamos el **RMSE** (raíz del error cuadrático medio) y el **MSE** (error cuadrático medio)  de nuestro modelo. 

In [44]:
rmseGBT= evaluator.evaluate(testGBT)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmseGBT)

Root Mean Squared Error (RMSE) on test data = 6694.64


In [45]:
mseGBT = rmseGBT*rmseGBT
print("Mean Squared Error (MSE) on test data = %g" % mseGBT) 

Mean Squared Error (MSE) on test data = 4.48182e+07


Podemos observar que el modelo de GRADIENT BOOSTED TREE REGRESOR nos da una predicción mucho mejor que RANDOM FOREST, pero nos continua dando un error cuadratico medio alto. Con esto podemos decir que **NO** es un buen método para nuestros datos 

___

## <span style="font-family:monospace;color:orchid">LINEAR REGRESSION</span>

In [46]:
from pyspark.ml.regression import LinearRegression

**¿Es _Linear Regression_ un algoritmo apropiado para nuestros datos?**

![.](https://media.geeksforgeeks.org/wp-content/uploads/python-linear-regression-4.png)

Linear regression es una técnica estadística donde podemos identificar que variables independientes (causas) explican una variable dependiente (resultado).

Vamos a comenzar creando una columna que reúna todas las variables utilizando el transformador **VectorAssembler**. A esta columna la llamaremos _'rawFeatures'_

In [47]:
mivectorLR = VectorAssembler(inputCols=featuresCols, outputCol="rawFeatures")

In [48]:
# This identifies categorical features and indexes them.
vectorIndexerLR = VectorIndexer(inputCol="rawFeatures", outputCol="features", maxCategories=4)

Montamos el **modelo** de Linear Regresision para predecir la columna 'price'

In [49]:
# Takes the "features" column and learns to predict "price_cluster"
lr = LinearRegression(labelCol="price")

Al igual que los casos anteriores, creamos el **pipeline**  para ejecutar nuestro VectorAssembler, el VectorIndexer y el modelo uno detrás de otro.

In [50]:
pipelineLR = Pipeline(stages=[mivectorLR, vectorIndexerLR, lr])

Realizamos el **modelado** del 80% de los datos y **entrenamos** el modelo con el 20% restante

In [51]:
modelLR = pipelineLR.fit(trainingData)

In [52]:
testLR = modelLR.transform(testData)

**Visualizamos** las tres primeras prediciones para ver como ha funcionado nuestro modelo

In [53]:
testLR.take(3)

[Row(price=2050, city=5, year=1999.0, manufacturer=3, cylinders=6, fuel=1, odometer=202000.0, transmission=1.0, drive=2, type=1, paint_color=2, rawFeatures=DenseVector([2050.0, 5.0, 1999.0, 3.0, 6.0, 1.0, 202000.0, 1.0, 2.0, 1.0, 2.0]), features=DenseVector([2050.0, 5.0, 1999.0, 3.0, 6.0, 1.0, 202000.0, 1.0, 1.0, 1.0, 2.0]), prediction=2050.0000000006953),
 Row(price=2100, city=2, year=1996.0, manufacturer=1, cylinders=6, fuel=1, odometer=151235.29235237173, transmission=1.0, drive=1, type=1, paint_color=2, rawFeatures=DenseVector([2100.0, 2.0, 1996.0, 1.0, 6.0, 1.0, 151235.2924, 1.0, 1.0, 1.0, 2.0]), features=DenseVector([2100.0, 2.0, 1996.0, 1.0, 6.0, 1.0, 151235.2924, 1.0, 0.0, 1.0, 2.0]), prediction=2100.000000000953),
 Row(price=2100, city=2, year=1999.0, manufacturer=5, cylinders=4, fuel=1, odometer=162695.9298187809, transmission=1.0, drive=1, type=6, paint_color=1, rawFeatures=DenseVector([2100.0, 2.0, 1999.0, 5.0, 4.0, 1.0, 162695.9298, 1.0, 1.0, 6.0, 1.0]), features=DenseVect

**Evaluamos** la precisión de predicción de nuestro modelo con un _RegressionEvaluator_. Al ver que  las predicciones de _linear regression_ son bastante parecidas a las originales podemos suponer que el error cuadrático medio será muy bajo.

In [54]:
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="price", predictionCol="prediction", metricName="rmse")
rmseLR = evaluator.evaluate(testLR)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmseLR)

Root Mean Squared Error (RMSE) on test data = 3.88653e-10


Calculamos el **MSE** (error cuadrático medio) de nuestro modelo.

In [55]:
mseLR = rmseLR*rmseLR
print("Mean Squared Error (MSE) on test data = %g" % mseLR) 

Mean Squared Error (MSE) on test data = 1.51051e-19


Viendo que el error cuadrático medio es prácticamente nulo, podemos confirmar que Linear Regression **SI** es un buen modelo para predecir nuestros datos

___

## <span style="font-family:monospace;color:deeppink">GRID SEARCH RANDOM FOREST</span>

Vamos a hacer un grid search de random forest para ver que parametros son los mejores para la prediccion de datos.Grid search es el proceso de escanear los datos para configurar parámetros óptimos para un modelo dado. Grid-Search creará un modelo en cada combinación de parámetros posible. Recorre cada combinación de parámetros y almacena un modelo para cada combinación.

In [56]:
import pyspark.ml.tuning as tune

forest =  rg.RandomForestRegressor(labelCol='price')

grid = tune.ParamGridBuilder().addGrid(forest.numTrees, [2, 5, 10, 15]).addGrid(forest.maxDepth, [5, 3, 2, 7]).build()

In [57]:
evaluator =RegressionEvaluator(predictionCol='prediction', labelCol='price')

Creamos la lógica que realizará la **validación**. El modelo itera através del grid de valores, estima los modelos y comparar su rendimiento utilizando el "evaluator"

In [58]:
rgC = tune.CrossValidator(estimator=forest, estimatorParamMaps=grid, evaluator=evaluator,numFolds=2)

In [59]:
pipeline = Pipeline(stages=[mivectorRF])
data_transformer = pipeline.fit(trainingData)

In [60]:
data_transformer = pipeline.fit(trainingData)

In [61]:
rgModel = rgC.fit(data_transformer.transform(trainingData))

In [62]:
data_train = data_transformer.transform(testData)

In [63]:
results = rgModel.transform(data_train)

In [64]:
results.take(1)

[Row(price=2050, city=5, year=1999.0, manufacturer=3, cylinders=6, fuel=1, odometer=202000.0, transmission=1.0, drive=2, type=1, paint_color=2, features=DenseVector([5.0, 1999.0, 3.0, 6.0, 1.0, 202000.0, 1.0, 2.0, 1.0, 2.0]), prediction=4040.8714597215762)]

In [65]:
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="price", predictionCol="prediction", metricName="rmse")
rmseGS = evaluator.evaluate(results)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmseGS)

Root Mean Squared Error (RMSE) on test data = 10122.1


In [66]:
mseGS = rmseGS*rmseGS
print("Mean Squared Error (MSE) on test data = %g" % mseGS) 

Mean Squared Error (MSE) on test data = 1.02457e+08


Podemos observar que, aunque tenga los parametros más adecuados, el error de predicción sigue siendo muy alto, por lo que esto nos confirma que Random Forest **NO** es el método adecuado para nuestros datos

In [67]:
results = [
    (
        [
            {key.name: paramValue} 
            for key, paramValue 
            in zip(
                params.keys(), 
                params.values())
        ], metric
    ) 
    for params, metric 
    in zip(
        rgModel.getEstimatorParamMaps(), 
        rgModel.avgMetrics
    )
]

sorted(results, key=lambda el: el[1], reverse=True)[0]

([{'numTrees': 2}, {'maxDepth': 2}], 10020.538996650168)

___

![.](https://civilian.com/wp-content/uploads/2020/04/Campaign_Measurement.png)

## <span style="font-family:monospace;color:crimson">DISCRETIZAR VARIABLES CONTINUAS </span>

Discretizar datos quiere decir convertir variables que son continuas en variables agrupadas por intervalos. Esta operación simplifica la información agrupando los objetos geográficos que presentan las mismas características en distintas clases. Hemos discretizado la columna de Price, ya que esta al ser una medición es una variable contínua.

In [68]:
x = np.arange(0, 100)
x = x / 100.0 * np.pi * 4
y = x * np.sin(x / 1.764) + 20.1234

schema = typ.StructType([
    typ.StructField('price', 
                    typ.DoubleType(), 
                    False
   )
])

data = sqlContext.createDataFrame([[float(e), ] for e in y], schema=schema)
#data = spark.createDataFrame([[float(e), ] for e in y], schema=schema)

Vamos a utilizar el _QuantileDiscretizer_ para partir nuestra variable continua en 5 buckets

In [69]:
discretizer = ft.QuantileDiscretizer(
    numBuckets=5, 
    inputCol='price', 
    outputCol='discretized')

In [70]:
data_discretized = discretizer.fit(cochesnuevo).transform(cochesnuevo)

data_discretized \
    .groupby('discretized')\
    .mean('price')\
    .sort('discretized')\
    .collect()

[Row(discretized=0.0, avg(price)=3715.7660397764826),
 Row(discretized=1.0, avg(price)=6998.078737565828),
 Row(discretized=2.0, avg(price)=11546.692363325943),
 Row(discretized=3.0, avg(price)=17883.877535947013),
 Row(discretized=4.0, avg(price)=32990.87914754255)]

## <span style="font-family:monospace;color:red">LINEAR REGRESSION DISCRETIZED</span>

Como *Regresión Lineal* es el algoritmo que mejor predicción nos ha dado, vamos a utilizarlo con los datos discretizados que acabamos de hallar.

Al tener nuevos datos, debemos volver a separar nuestros datos en entrenamiento y test en un 80-20

In [71]:
(trainingDataD, testDataD) = data_discretized.randomSplit([0.8, 0.2])

Realizamos los MISMOS PASOS que Linear Regressionn

In [72]:
modelLRD = pipelineLR.fit(trainingDataD)

In [73]:
testLRD = modelLRD.transform(testDataD)

In [74]:
testLRD.take(3)

[Row(price=2003.0, city=6, year=2003.0, manufacturer=4, cylinders=6, fuel=1, odometer=120000.0, transmission=1.0, drive=2, type=1, paint_color=2, discretized=0.0, rawFeatures=DenseVector([2003.0, 6.0, 2003.0, 4.0, 6.0, 1.0, 120000.0, 1.0, 2.0, 1.0, 2.0]), features=DenseVector([2003.0, 6.0, 2003.0, 4.0, 6.0, 1.0, 120000.0, 1.0, 1.0, 1.0, 2.0]), prediction=2002.9999999995505),
 Row(price=2013.0, city=6, year=2013.0, manufacturer=14, cylinders=4, fuel=1, odometer=98895.41200787401, transmission=2.0, drive=2, type=1, paint_color=2, discretized=0.0, rawFeatures=DenseVector([2013.0, 6.0, 2013.0, 14.0, 4.0, 1.0, 98895.412, 2.0, 2.0, 1.0, 2.0]), features=DenseVector([2013.0, 6.0, 2013.0, 14.0, 4.0, 1.0, 98895.412, 2.0, 1.0, 1.0, 2.0]), prediction=2013.0000000002572),
 Row(price=2075.0, city=32, year=2003.0, manufacturer=5, cylinders=6, fuel=1, odometer=153430.5303265941, transmission=1.0, drive=1, type=1, paint_color=2, discretized=0.0, rawFeatures=DenseVector([2075.0, 32.0, 2003.0, 5.0, 6.0, 

In [75]:
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="price", predictionCol="prediction", metricName="rmse")
rmseLRD = evaluator.evaluate(testLRD)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmseLRD)

Root Mean Squared Error (RMSE) on test data = 6.72951e-10


In [76]:
mseLRD = rmseLRD*rmseLRD
print("Mean Squared Error (MSE) on test data = %g" % mseLRD) 

Mean Squared Error (MSE) on test data = 4.52862e-19


Podemos observar, que con datos discretizados, el error de predicción con _Logistic Regression_ sigue siendo una buena opción para nuestros datos

___

## <span style="font-family:monospace;color:tomato">ESTANDARIZAR VARIABLES CONTINUAS</span>

La estandarización o normalización de índices significa ajustar los valores medidos en diferentes escalas respecto a una escala común, a menudo previo a un proceso de realizar promedios.

In [77]:
vectorizer = ft.VectorAssembler(inputCols=['price'], outputCol= 'price_vec')

Construimos el "normalizer" y el "pipeline". Pondremos "withMean=True" y "withStd=True" para que la media y la varianza sean de un longitud de uno.

In [78]:
normalizer = ft.StandardScaler(
    inputCol=vectorizer.getOutputCol(), 
    outputCol='normalized', 
    withMean=True,
    withStd=True
)

In [79]:
pipeline = Pipeline(stages=[vectorizer, normalizer])
data_standardized = pipeline.fit(cochesnuevo).transform(cochesnuevo)

In [80]:
data_standardized.take(1)

[Row(price=9000.0, city=1, year=2009.0, manufacturer=2, cylinders=8, fuel=1, odometer=217743.0, transmission=1.0, drive=3, type=2, paint_color=2, price_vec=DenseVector([9000.0]), normalized=DenseVector([-0.4383]))]

## <span style="font-family:monospace;color:lightsalmon">LINEAR REGRESSION STANDARIZED</span>

Al igual que en el caso anterior, vamos a utilizar *Regresión Lineal* ya que es el algoritmo que mejor predicción nos ha dado, por lo vamos a utilizarlo con los datos estandarizados que acabamos de hallar.

In [86]:
(trainingDataE, testDataE) = data_discretized.randomSplit([0.8, 0.2])

Realizamos los MISMOS PASOS que Linear Regressionn

In [87]:
modelLRE = pipelineLR.fit(trainingDataE)

In [88]:
testLRE = modelLRE.transform(testDataE)

In [89]:
testLRE.take(3)

[Row(price=2003.0, city=6, year=2003.0, manufacturer=4, cylinders=6, fuel=1, odometer=120000.0, transmission=1.0, drive=2, type=1, paint_color=2, discretized=0.0, rawFeatures=DenseVector([2003.0, 6.0, 2003.0, 4.0, 6.0, 1.0, 120000.0, 1.0, 2.0, 1.0, 2.0]), features=DenseVector([2003.0, 6.0, 2003.0, 4.0, 6.0, 1.0, 120000.0, 1.0, 1.0, 1.0, 2.0]), prediction=2002.999999999999),
 Row(price=2050.0, city=7, year=2003.0, manufacturer=6, cylinders=8, fuel=1, odometer=175500.0, transmission=1.0, drive=3, type=4, paint_color=9, discretized=0.0, rawFeatures=DenseVector([2050.0, 7.0, 2003.0, 6.0, 8.0, 1.0, 175500.0, 1.0, 3.0, 4.0, 9.0]), features=DenseVector([2050.0, 7.0, 2003.0, 6.0, 8.0, 1.0, 175500.0, 1.0, 2.0, 4.0, 9.0]), prediction=2050.000000000008),
 Row(price=2050.0, city=23, year=2000.0, manufacturer=17, cylinders=6, fuel=1, odometer=156888.25572112825, transmission=1.0, drive=1, type=1, paint_color=2, discretized=0.0, rawFeatures=DenseVector([2050.0, 23.0, 2000.0, 17.0, 6.0, 1.0, 156888.2

In [90]:
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="price", predictionCol="prediction", metricName="rmse")
rmseLRE = evaluator.evaluate(testLRE)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmseLRE)

Root Mean Squared Error (RMSE) on test data = 5.20874e-12


In [91]:
mseLRE = rmseLRE*rmseLRE
print("Mean Squared Error (MSE) on test data = %g" % mseLRE) 

Mean Squared Error (MSE) on test data = 2.7131e-23


Podemos observar que con datos estandarizados el error de predicción sigue siendo casi nulo, por lo que este método es muy útil para nuestros datos

___

## <span style="font-family:monospace;color:seagreen">PCA</span>

![.](https://i2.wp.com/www.sportscidata.com/wp-content/uploads/2019/08/Principal_Component_Analysis_print.png?fit=1024%2C683&ssl=1)

PCA es una técnica para resaltar patrones fuertes de un conjunto de datos. Este método se utiliza a menudo para facilitar la visualización y exploración de los datos.

In [92]:
from pyspark.ml.feature import PCA
from pyspark.ml.feature import ChiSqSelector

In [93]:
featuresCreatorPCA = ft.VectorAssembler(
    inputCols=["city","year","manufacturer","cylinders","fuel","odometer","transmission","drive","type","paint_color"], 
    outputCol='features'
)

In [94]:
from pyspark.ml.feature import PCA
pca = PCA(k=2, inputCol='features', outputCol='pcaFeature')
algoritmoPCA = rg.LinearRegression(labelCol="price")
pipeline = Pipeline(stages=[featuresCreatorPCA,pca,algoritmoPCA]) #pca

In [95]:
model = pipeline.fit(trainingData)
prediction = model.transform(testData)
prediction.show()

+-----+----+------+------------+---------+----+------------------+------------+-----+----+-----------+--------------------+--------------------+-------------------+
|price|city|  year|manufacturer|cylinders|fuel|          odometer|transmission|drive|type|paint_color|            features|          pcaFeature|         prediction|
+-----+----+------+------------+---------+----+------------------+------------+-----+----+-----------+--------------------+--------------------+-------------------+
| 2050|   5|1999.0|           3|        6|   1|          202000.0|         1.0|    2|   1|          2|[5.0,1999.0,3.0,6...|[201999.989796409...|-1448.9845224986784|
| 2100|   2|1996.0|           1|        6|   1|151235.29235237173|         1.0|    1|   1|          2|[2.0,1996.0,1.0,6...|[151235.282167249...| -3131.806385799311|
| 2100|   2|1999.0|           5|        4|   1| 162695.9298187809|         1.0|    1|   6|          1|[2.0,1999.0,5.0,4...|[162695.919615336...| -2593.600738449022|
| 2100|   

In [96]:
prediction.head()

Row(price=2050, city=5, year=1999.0, manufacturer=3, cylinders=6, fuel=1, odometer=202000.0, transmission=1.0, drive=2, type=1, paint_color=2, features=DenseVector([5.0, 1999.0, 3.0, 6.0, 1.0, 202000.0, 1.0, 2.0, 1.0, 2.0]), pcaFeature=DenseVector([201999.9898, 1.1736]), prediction=-1448.9845224986784)

Podemos observar a continuación que la columna que es mejor para la predicción de price es la de **odometer** o cuentakilómetros.

In [97]:
prediction.select('pcaFeature').distinct().show()

+--------------------+
|          pcaFeature|
+--------------------+
|[147984.503646030...|
|[118.989734965029...|
|[211126.989813806...|
|[106999.989786061...|
|[152999.989765642...|
|[132699.989756483...|
|[157218.966576743...|
|[145999.989732149...|
|[114504.989746453...|
|[153444.989751703...|
|[128914.026669885...|
|[136999.989740296...|
|[137831.989749953...|
|[246522.989768092...|
|[117999.989736395...|
|[134729.974814199...|
|[112388.989743027...|
|[125190.989731532...|
|[299998.989763416...|
|[138314.989749604...|
+--------------------+
only showing top 20 rows



In [98]:
prediction.select('features').distinct().show()

+--------------------+
|            features|
+--------------------+
|[3.0,2005.0,2.0,4...|
|[10.0,2001.0,30.0...|
|[23.0,2003.0,8.0,...|
|[10.0,2004.0,7.0,...|
|[2.0,2006.0,20.0,...|
|[2.0,2002.0,1.0,4...|
|[15.0,2000.0,3.0,...|
|[24.0,2006.0,2.0,...|
|[44.0,2006.0,12.0...|
|[20.0,2009.0,1.0,...|
|[2.0,2000.0,1.0,6...|
|[2.0,2006.0,2.0,8...|
|[9.0,2007.0,9.0,4...|
|[1.0,2007.0,4.0,4...|
|[19.0,2005.0,14.0...|
|[3.0,2005.0,19.0,...|
|[20.0,2001.0,4.0,...|
|[10.0,2011.0,2.0,...|
|[1.0,2007.0,30.0,...|
|[9.0,2003.0,2.0,8...|
+--------------------+
only showing top 20 rows



___

![.](https://www.giitsss.com/public_assets/img/result_banner_giit.jpg)

## <span style="font-family:monospace;color:mediumaquamarine">COMPARACIÓN FINAL DE LOS MÉTODOS</span>

In [123]:
print("RANDOM FOREST: MSE = %g" % mseRF + ": 💩") 
print("RANDOM FOREST + GRID SEARCH: MSE = %g" % mseGS + ": 😫")
print("GRADIENT BOOSTED TREE REGRESSOR: MSE = %g" % mseGBT + ": ☹️")
print("LINEAR REGRESSION + DISCRETIZADOS:MSE = %g" % mseLRD + ": 🙂") 
print("LINEAR REGRESSION: MSE = %g" % mseLR + ": 😀")
print("LINEAR REGRESSION + ESTANDARIZADOS: MSE = %g" % mseLRE + ": 👑") 

RANDOM FOREST: MSE = 1.15632e+08: 💩
RANDOM FOREST + GRID SEARCH: MSE = 1.02457e+08: 😫
GRADIENT BOOSTED TREE REGRESSOR: MSE = 4.48182e+07: ☹️
LINEAR REGRESSION + DISCRETIZADOS:MSE = 4.52862e-19: 🙂
LINEAR REGRESSION: MSE = 1.51051e-19: 😀
LINEAR REGRESSION + ESTANDARIZADOS: MSE = 2.7131e-23: 👑


___

![.](https://www.vinilosdecorativos.com/3998/fotomural-infantil-rayo-mcqueen-y-mate-ruta-por-el-desierto.jpg)

# <span style="font-family:monospace;color:dodgerblue">CONCLUSIONES</span> 

                                            ¿QUÉ HEMOS APRENDIDO?

___

# <span style="font-family:impact;font-size:3em;color:tomato">Y</span>   <span style="font-family:impact;font-size:3em;color:coral">P</span><span style="font-family:impact;font-size:3em;color:salmon">A</span><span style="font-family:impact;font-size:3em;color:lightgreen">R</span><span style="font-family:impact;font-size:3em;color:limegreen">A</span>    <span style="font-family:impact;font-size:3em;color:mediumseagreen">F</span><span style="font-family:impact;font-size:3em;color:turquoise">I</span><span style="font-family:impact;font-size:3em;color:darkturquoise">N</span><span style="font-family:impact;font-size:3em;color:deepskyblue">A</span><span style="font-family:impact;font-size:3em;color:cornflowerblue">L</span><span style="font-family:impact;font-size:3em;color:royalblue">I</span><span style="font-family:impact;font-size:3em;color:slateblue">Z</span><span style="font-family:impact;font-size:3em;color:mediumorchid">A</span><span style="font-family:impact;font-size:3em;color:violet">R</span>   <span style="font-family:impact;font-size:3em;color:hotpink">...</span>

![.](https://www.generadormemes.com/media/created/oms18kkaw3nfr12msw8m5b3w5nbywfxcr7logoz2c7quikjn0dvx20q6azucxil.jpg)